### Import Libs

In [24]:
# Adding needed libraries and reading data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from databricks import koalas as ks
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Load Data

In [55]:
kdf_train =  ks.from_pandas(pd.read_csv('train.csv'))
kdf_test =  ks.from_pandas(pd.read_csv('test.csv'))
print(kdf_train.shape)
print(kdf_test.shape)

(1460, 81)
(1459, 80)


### Drop Columns

In [56]:
train_labels = kdf_train.pop('SalePrice')
features = ks.concat([kdf_train, kdf_test])
features = features.drop(['Utilities', 'RoofMatl', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'LowQualFinSF',
               'BsmtFullBath', 'BsmtHalfBath', 'Functional', 'GarageYrBlt', 'GarageArea', 'GarageCond', 'WoodDeckSF',
               'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal'],
              axis=1)
print(features.shape)

(2919, 56)


### FillNa & Converting

In [58]:
# MSSubClass as str
features['MSSubClass'] = features['MSSubClass'].astype(str)

# MSZoning NA in pred. filling with most popular values
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# LotFrontage  NA in all. I suppose NA means 0
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())

# Alley  NA in all. NA means no access
features['Alley'] = features['Alley'].fillna('NOACCESS')

# Converting OverallCond to str
features.OverallCond = features.OverallCond.astype(str)

# MasVnrType NA in all. filling with most popular values
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# NA in all. NA means No basement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# TotalBsmtSF  NA in pred. I suppose NA means 0
features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# Electrical NA in pred. filling with most popular values
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

# KitchenAbvGr to categorical
features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# KitchenQual NA in pred. filling with most popular values
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# FireplaceQu  NA in all. NA means No Fireplace
features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

# GarageType, GarageFinish, GarageQual  NA in all. NA means No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# GarageCars  NA in pred. I suppose NA means 0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# SaleType NA in pred. filling with most popular values
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

# Year and Month to categorical
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

# Adding total sqfootage feature and removing Basement, 1st and 2nd floor features
features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features = features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1)

### Log transformation

In [63]:
## Standardizing numeric features
numeric_features = features.loc[:,['LotFrontage', 'LotArea', 'GrLivArea', 'TotalSF']]
#numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

### Converting categorical data to dummies

In [66]:
# # Getting Dummies from Condition1 and Condition2
# conditions = set([x for x in features['Condition1']] + [x for x in features['Condition2']])
# dummies = ks.DataFrame(data=np.zeros((len(features.index), len(conditions))),
#                        index=features.index, columns=conditions)
# for i, cond in enumerate(zip(features['Condition1'], features['Condition2'])):
#     dummies.ix[i, cond] = 1
# features = ks.concat([features, dummies.add_prefix('Condition_')], axis=1)
# features = features.drop(['Condition1', 'Condition2'], axis=1)

# # Getting Dummies from Exterior1st and Exterior2nd
# exteriors = set([x for x in features['Exterior1st']] + [x for x in features['Exterior2nd']])
# dummies = ks.DataFrame(data=np.zeros((len(features.index), len(exteriors))),
#                        index=features.index, columns=exteriors)
# for i, ext in enumerate(zip(features['Exterior1st'], features['Exterior2nd'])):
#     dummies.ix[i, ext] = 1
# features = ks.concat([features, dummies.add_prefix('Exterior_')], axis=1)
# features = features.drop(['Exterior1st', 'Exterior2nd', 'Exterior_nan'], axis=1)

# # Getting Dummies from all other categorical vars
# for col in features.dtypes[features.dtypes == 'object'].index:
#     for_dummy = features.pop(col)
#     features = pd.concat([features, pd.get_dummies(for_dummy, prefix=col)], axis=1)